# Лабораторная работа 1

По заданным параметрам составьте модель линейного программирования оптимального финансирования проектов для следующей задачи.   
Рассматривается задача финансирования проектов. Имеется набор проектов и набор источников, также предполагается наличие собственных средств в начальный момент времени. По каждому проекту и источнику известны движения денежных средств (денежные потоки) на заданном количестве периодов.

Переменные - доля вложения в каждый из проектов и доля использования каждого из источников. 
1. Генерация начальных данных (python) (ИИ: создание 50 тестов, ИБ: один тест):

    1.1     Кратны 5, минимум: 5 проектов, 5 источников, 4 периода, собственные средства.

    1.2     Для каждого проекта или источника есть хотя бы один период положительного денежного потока и отрицательного.
 (например, по таблице, для проекта 1 период 0 - отрицательный, периоды 1, 2, 3 - положительные)

    1.3     Для проектов: нет заведомо невыгодных проектов: сумма денежных потоков больше 0 и  доход не менее чем на 10% больше от всех вложений (расходов) в проект. (например, по таблице,  для проекта 1 расходы: 50, доходы: 60, то есть доходы на 20% больше, все ок)

    1.4     Для источников: нет источников, которые не получают прибыль: сумма денежных потоков каждого источника меньше нуля и доходность каждого источника не менее 15% от изначальных вложений. 

    1.5     Полузакрытая система: сумма вложений в проекты больше равна сумме доходов от всех источников финансирования, включая собственные средства.  (например, по таблице,  для проектов вложения 50 + 150 + 100 (отрицтальные величины) = 300, средства от источников (положительные величины) 100 + 50 + 50 = 200; 300 >= 200 все ок)

2. Самостоятельный выбор целевой функции (с объяснением) и построение мат модели с конкретными числами на одной из задач (тестов). 

3. Решение с помощью библиотек scipy и pulp.

4. Сравнение библиотек по времени и памяти (дополнительно для ИИ: визуализация сравнения).

In [18]:
import numpy as np
import pandas as pd

# Параметры задачи
num_projects: int = 5  # Количество проектов
num_sources: int = 5  # Количество источников финансирования
num_periods: int = 4  # Количество периодов

# Фиксируем seed для воспроизводимости
np.random.seed(42)


# Функция для генерации потоков денежных средств
def generate_cash_flows(min_value: int, max_value: int, num_periods: int) -> np.ndarray:
    """
    Генерирует потоки денежных средств с хотя бы одним положительным и одним отрицательным значением.

    :param min_value: Минимальное значение в потоке
    :param max_value: Максимальное значение в потоке
    :param num_periods: Количество периодов
    :return: Массив сгенерированных данных
    """
    cash_flows = np.random.choice(
        np.arange(min_value, max_value + 1, 5), num_periods, replace=True
    )

    # Проверяем, есть ли хотя бы одно положительное и одно отрицательное значение
    if np.all(cash_flows >= 0) or np.all(cash_flows <= 0):
        pos_idx = (
            np.random.choice(np.where(cash_flows > 0)[0], 1)
            if np.any(cash_flows > 0)
            else np.random.choice(num_periods, 1)
        )
        neg_idx = (
            np.random.choice(np.where(cash_flows < 0)[0], 1)
            if np.any(cash_flows < 0)
            else np.random.choice(num_periods, 1)
        )
        cash_flows[pos_idx] = abs(cash_flows[pos_idx])
        cash_flows[neg_idx] = -abs(cash_flows[neg_idx])

    return cash_flows


# Проверка выполнения инвестиционного условия
def validate_investment_condition(
    projects: np.ndarray, sources: np.ndarray, own_funds: float,
) -> bool:
    """
    Проверяет условие, что сумма вложений в проекты больше или равна сумме доходов от всех источников финансирования, включая собственные средства.

    :param projects: Массив денежных потоков по проектам
    :param sources: Массив денежных потоков по источникам
    :param own_funds: Собственные средства
    :return: True, если условие выполняется, иначе False
    """
    total_project_investment = abs(
        projects[projects < 0].sum()
    )  # Вложения в проекты (отрицательные значения)
    total_source_income = (
        sources[sources > 0].sum() + own_funds
    )  # Доходы от источников и собственные средства (только в периоде 0)
    return total_project_investment >= total_source_income


projects: np.ndarray = np.zeros((num_projects, num_periods))
sources: np.ndarray = np.zeros((num_sources, num_periods))
own_funds: float = np.random.randint(50, 100)  # Собственные средства на период 0

# Генерируем проекты и источники поэтапно
for i in range(num_projects):
    while True:
        project_cash_flows = generate_cash_flows(-200, 200, num_periods)
        if (
            project_cash_flows.sum()
            > abs(project_cash_flows[project_cash_flows < 0].sum()) * 1.1
        ):
            projects[i, :] = project_cash_flows
            break

# Для источников и собственных средств будем следить за выполнением условия
while True:
    for i in range(num_sources):
        while True:
            source_cash_flows = generate_cash_flows(-150, 150, num_periods)
            if (
                source_cash_flows.sum() < 0
                and abs(source_cash_flows[source_cash_flows > 0].sum())
                >= abs(source_cash_flows.sum()) * 1.15
            ):
                sources[i, :] = source_cash_flows
                break

    # Проверка инвестиционного условия с учетом собственных средств в периоде 0
    if validate_investment_condition(projects, sources, own_funds):
        break  


columns = (
    ["Период"]
    + [f"Проект {i+1}" for i in range(num_projects)]
    + [f"Источник {i+1}" for i in range(num_sources)]
    + ["Собственные средства"]
)
data = np.hstack([np.arange(num_periods).reshape(-1, 1), projects.T, sources.T])

# Добавляем собственные средства в период 0
data_with_own_funds = np.column_stack([data, [own_funds] + [0] * (num_periods - 1)])

# Создаем DataFrame для отображения данных
df_cashflows = pd.DataFrame(data_with_own_funds, columns=columns)

# Выводим таблицу с денежными потоками
print(df_cashflows)

   Период  Проект 1  Проект 2  Проект 3  Проект 4  Проект 5  Источник 1  \
0     0.0      55.0      85.0       5.0     105.0     -30.0         0.0   
1     1.0    -130.0     -95.0      95.0     105.0     185.0       -10.0   
2     2.0     155.0      40.0     195.0      30.0     200.0        20.0   
3     3.0     100.0      90.0    -130.0    -105.0     -25.0       -15.0   

   Источник 2  Источник 3  Источник 4  Источник 5  Собственные средства  
0        20.0       -75.0        20.0       -40.0                  88.0  
1      -130.0       145.0       -30.0       -25.0                   0.0  
2         0.0        15.0       -15.0       125.0                   0.0  
3        55.0       -90.0        10.0      -110.0                   0.0  
